In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pydicom as dicom
import pandas as pd
import numpy as np
from random import randint

import os, sys, time, shutil, scipy, cv2, json, datetime
import numpy as np
import pandas as pd
import PIL.Image
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from tqdm.notebook import tqdm

# ignoring warnings
import warnings
warnings.simplefilter("ignore")

import torch
from torch.utils.data import TensorDataset, DataLoader,Dataset
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data.sampler import SubsetRandomSampler, RandomSampler, SequentialSampler
from torch.optim.lr_scheduler import CosineAnnealingLR
from warmup_scheduler import GradualWarmupScheduler 
import albumentations as A
import geffnet

from scipy import ndimage
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
#Import from keras_preprocessing not from keras.preprocessing
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error, r2_score, roc_auc_score
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers, optimizers

# Build a baseline fully connected model
from keras import models
from keras import layers

# Dataset 1 (Train and Validation)

In [2]:
df = pd.read_csv('data/ISIC_2020_Training_GroundTruth_v2.csv')
df.columns = ['file', 'patient_id', 'lesion_id', 'gender', 'age', 'site', 'diagnosis', 'ben_mal', 'target']
df['jpg'] = df['file'].apply(lambda x: str(x) + ".jpg")
df.head()

,file,patient_id,lesion_id,gender,age,site,diagnosis,ben_mal,target,jpg
0,ISIC_2637011,IP_7279968,IL_7972535,male,45.0,head/neck,unknown,benign,0,ISIC_2637011.jpg
1,ISIC_0015719,IP_3075186,IL_4649854,female,45.0,upper extremity,unknown,benign,0,ISIC_0015719.jpg
2,ISIC_0052212,IP_2842074,IL_9087444,female,50.0,lower extremity,nevus,benign,0,ISIC_0052212.jpg
3,ISIC_0068279,IP_6890425,IL_4255399,female,45.0,head/neck,unknown,benign,0,ISIC_0068279.jpg
4,ISIC_0074268,IP_8723313,IL_6898037,female,55.0,upper extremity,unknown,benign,0,ISIC_0074268.jpg


In [3]:
melanoma = df[df.target == 1]
non_melanoma = df[df.target == 0]
y_mel = melanoma.target
X_mel = melanoma.drop(columns=['target'], axis=1)
y_non_mel = non_melanoma.target
X_non_mel = non_melanoma.drop(columns=['target'], axis=1)

In [4]:


X_tr_m, X_val_m, y_tr_m, y_val_m = train_test_split(X_mel, y_mel, random_state=42, test_size=0.2)
X_tr_nm, X_val_nm, y_tr_nm, y_val_nm = train_test_split(X_non_mel, y_non_mel, random_state=42, test_size=0.2)

mel_train = pd.concat([X_tr_m, y_tr_m], axis=1)
non_mel_train = pd.concat([X_tr_nm, y_tr_nm], axis=1)
mel_val = pd.concat([X_val_m, y_val_m], axis=1)
non_mel_val = pd.concat([X_val_nm, y_val_nm], axis=1)

train = pd.concat([mel_train, non_mel_train], axis=0)
val = pd.concat([mel_val, non_mel_val], axis=0)
train = train.reset_index(drop=True)
val = val.reset_index(drop=True)

In [19]:
train.head()

,file,patient_id,lesion_id,gender,age,site,diagnosis,ben_mal,jpg,target
0,ISIC_1569119,IP_6120178,IL_3157235,male,65.0,torso,melanoma,malignant,ISIC_1569119.jpg,1
1,ISIC_7536704,IP_3994607,IL_1685088,male,75.0,torso,melanoma,malignant,ISIC_7536704.jpg,1
2,ISIC_3696488,IP_6051587,IL_8878499,female,90.0,upper extremity,melanoma,malignant,ISIC_3696488.jpg,1
3,ISIC_3343475,IP_4576785,IL_9262145,male,80.0,head/neck,melanoma,malignant,ISIC_3343475.jpg,1
4,ISIC_7785723,IP_8447624,IL_2711936,female,50.0,lower extremity,melanoma,malignant,ISIC_7785723.jpg,1


In [21]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26500 entries, 0 to 26499
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   file        26500 non-null  object 
 1   patient_id  26500 non-null  object 
 2   lesion_id   26500 non-null  object 
 3   gender      26449 non-null  object 
 4   age         26447 non-null  float64
 5   site        26088 non-null  object 
 6   diagnosis   26500 non-null  object 
 7   ben_mal     26500 non-null  object 
 8   jpg         26500 non-null  object 
 9   target      26500 non-null  int64  
dtypes: float64(1), int64(1), object(8)
memory usage: 2.0+ MB


In [20]:
val.head()

,file,patient_id,lesion_id,gender,age,site,diagnosis,ben_mal,jpg,target
0,ISIC_6767569,IP_7665112,IL_1769376,male,70.0,head/neck,melanoma,malignant,ISIC_6767569.jpg,1
1,ISIC_7312977,IP_8675890,IL_2723500,male,55.0,lower extremity,melanoma,malignant,ISIC_7312977.jpg,1
2,ISIC_1785627,IP_0170821,IL_6029631,female,40.0,upper extremity,melanoma,malignant,ISIC_1785627.jpg,1
3,ISIC_8066110,IP_3055814,IL_7066374,male,50.0,lower extremity,melanoma,malignant,ISIC_8066110.jpg,1
4,ISIC_8838753,IP_6292815,IL_9486623,female,65.0,lower extremity,melanoma,malignant,ISIC_8838753.jpg,1


In [22]:
val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6626 entries, 0 to 6625
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   file        6626 non-null   object 
 1   patient_id  6626 non-null   object 
 2   lesion_id   6626 non-null   object 
 3   gender      6612 non-null   object 
 4   age         6611 non-null   float64
 5   site        6511 non-null   object 
 6   diagnosis   6626 non-null   object 
 7   ben_mal     6626 non-null   object 
 8   jpg         6626 non-null   object 
 9   target      6626 non-null   int64  
dtypes: float64(1), int64(1), object(8)
memory usage: 517.8+ KB


# Dataset 2 (Test)

In [6]:
test1 = pd.read_csv('data/ISIC_2019_Training_GroundTruth.csv')
test2 = pd.read_csv('data/ISIC_2019_Training_Metadata.csv')
test1 = test1.rename(columns={'image':'image1'})
test2 = test2.rename(columns={'image':'image2'})
test = pd.concat([test2, test1], axis=1)
test = test.drop(columns=['image1'], axis=1)
test = test.drop(columns=['NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC', 'UNK'], axis=1)
test.columns = ['file', 'age', 'site', 'lesion_id', 'gender', 'target']
test.target = test.target.astype(int)
test['jpg'] = test.file.apply(lambda x: str(x) + ".jpg")
test.head()

,file,age,site,lesion_id,gender,target,jpg
0,ISIC_0000000,55.0,anterior torso,NaN,female,0,ISIC_0000000.jpg
1,ISIC_0000001,30.0,anterior torso,NaN,female,0,ISIC_0000001.jpg
2,ISIC_0000002,60.0,upper extremity,NaN,female,1,ISIC_0000002.jpg
3,ISIC_0000003,30.0,upper extremity,NaN,male,0,ISIC_0000003.jpg
4,ISIC_0000004,80.0,posterior torso,NaN,male,1,ISIC_0000004.jpg


In [23]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25331 entries, 0 to 25330
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   file       25331 non-null  object 
 1   age        24894 non-null  float64
 2   site       22700 non-null  object 
 3   lesion_id  23247 non-null  object 
 4   gender     24947 non-null  object 
 5   target     25331 non-null  int64  
 6   jpg        25331 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 1.4+ MB


# Proper Folder Structure

In [10]:
# assign folder variables
new_dir = 'split/'
tr_fldr = os.path.join(new_dir, 'train')
tr_mel = os.path.join(tr_fldr, 'mel')
tr_nm = os.path.join(tr_fldr, 'not_mel')
val_fldr = os.path.join(new_dir, 'val')
val_mel = os.path.join(val_fldr, 'mel')
val_nm = os.path.join(val_fldr, 'not_mel')
tt_fldr = os.path.join(new_dir, 'test')
tt_mel = os.path.join(tt_fldr, 'mel')
tt_nm = os.path.join(tt_fldr, 'not_mel')
images_dir = 'images/train_jpg'

In [22]:
# make new directories
os.mkdir(new_dir)
os.mkdir(tt_fldr)
os.mkdir(tt_mel)
os.mkdir(tt_nm)
os.mkdir(tr_fldr)
os.mkdir(tr_mel)
os.mkdir(tr_nm)
os.mkdir(val_fldr)
os.mkdir(val_mel)
os.mkdir(val_nm)

## Training Set

In [ ]:
mel_tr_ids = mel_train.jpg.to_list()
nm_tr_ids = non_mel_train.jpg.to_list()

In [12]:
# move images into train/mel folder
for img in mel_tr_ids:
    origin = os.path.join(images_dir, img)
    destination = os.path.join(tr_mel, img)
    shutil.copyfile(origin, destination)

In [30]:
len(nm_tr_ids)

26033

In [33]:
tr_1 = nm_tr_ids[0:2000]
tr_2 = nm_tr_ids[2000:4000]
tr_3 = nm_tr_ids[4000:6000]
tr_4 = nm_tr_ids[6000:8000]
tr_5 = nm_tr_ids[8000:10000]
tr_6 = nm_tr_ids[10000:12000]
tr_7 = nm_tr_ids[12000:14000]
tr_8 = nm_tr_ids[14000:16000]
tr_9 = nm_tr_ids[16000:18000]
tr_10 = nm_tr_ids[18000:20000]
tr_11 = nm_tr_ids[20000:]

In [1]:
# tr_lst = [tr_1, tr_2, tr_3, tr_4, tr_5, tr_6, tr_7, tr_8, tr_9, tr_10, tr_11]

# for lst in tqdm(tr_lst):
#     for img in lst:
#         shutil.copyfile(os.path.join(images_dir, img), os.path.join(tr_nm, img))

In [39]:
for img in tqdm(tr_1):
    shutil.copyfile(os.path.join(images_dir, img), os.path.join(tr_nm, img))

In [ ]:
for img in tqdm(tr_2):
    shutil.copyfile(os.path.join(images_dir, img), os.path.join(tr_nm, img))

In [ ]:
for img in tqdm(tr_3):
    shutil.copyfile(os.path.join(images_dir, img), os.path.join(tr_nm, img))

In [ ]:
for img in tqdm(tr_4):
    shutil.copyfile(os.path.join(images_dir, img), os.path.join(tr_nm, img))

In [ ]:
for img in tqdm(tr_5):
    shutil.copyfile(os.path.join(images_dir, img), os.path.join(tr_nm, img))

In [ ]:
for img in tqdm(tr_6):
    shutil.copyfile(os.path.join(images_dir, img), os.path.join(tr_nm, img))

In [ ]:
for img in tqdm(tr_7):
    shutil.copyfile(os.path.join(images_dir, img), os.path.join(tr_nm, img))

In [ ]:
for img in tqdm(tr_8):
    shutil.copyfile(os.path.join(images_dir, img), os.path.join(tr_nm, img))

In [ ]:
for img in tqdm(tr_9):
    shutil.copyfile(os.path.join(images_dir, img), os.path.join(tr_nm, img))

In [ ]:
for img in tqdm(tr_10):
    shutil.copyfile(os.path.join(images_dir, img), os.path.join(tr_nm, img))

In [ ]:
for img in tqdm(tr_11):
    shutil.copyfile(os.path.join(images_dir, img), os.path.join(tr_nm, img))

## Validation Set

In [ ]:
mel_val_ids = mel_val.jpg.to_list()
nm_val_ids = non_mel_val.jpg.to_list()

In [15]:
# move images into val/mel folder
for img in mel_val_ids:
    origin = os.path.join(images_dir, img)
    destination = os.path.join(val_mel, img)
    shutil.copyfile(origin, destination)    

In [27]:
# move images into val/non-mel folder
for img in nm_val_ids:
    origin = os.path.join(images_dir, img)
    destination = os.path.join(val_nm, img)
    shutil.copyfile(origin, destination)

## Testing Set

In [28]:
# move 
tt_mel_df = test[test.target == 1]
tt_nm_df = test[test.target == 0]
tt_mel_ids = tt_mel_df.jpg.to_list()
tt_nm_ids = tt_nm_df.jpg.to_list()
test_dir = 'images/test'


In [2]:
len(tt_nm_ids)

NameError: name 'tt_nm_ids' is not defined

In [29]:
for img in tt_mel_ids:
    origin = os.path.join(test_dir, img)
    destination = os.path.join(tt_mel, img)
    shutil.copyfile(origin, destination)

In [ ]:
for img in tt_nm_ids:
    origin = os.path.join(test_dir, img)
    destination = os.path.join(tt_nm, img)
    shutil.copyfile(origin, destination)

In [35]:
len(tt_nm_ids)

20809

In [36]:
tt_1 = tt_nm_ids[0:2000]
tt_2 = tt_nm_ids[2000:4000]
tt_3 = tt_nm_ids[4000:6000]
tt_4 = tt_nm_ids[6000:8000]
tt_5 = tt_nm_ids[8000:10000]
tt_6 = tt_nm_ids[10000:12000]
tt_7 = tt_nm_ids[12000:14000]
tt_8 = tt_nm_ids[14000:16000]
tt_9 = tt_nm_ids[16000:18000]
tt_10 = tt_nm_ids[18000:]


In [ ]:
for img in tqdm(tt_1):
    shutil.copyfile(os.path.join(test_dir, img), os.path.join(tt_nm, img))

In [ ]:
for img in tqdm(tt_2):
    shutil.copyfile(os.path.join(test_dir, img), os.path.join(tt_nm, img))

In [ ]:
for img in tqdm(tt_3):
    shutil.copyfile(os.path.join(test_dir, img), os.path.join(tt_nm, img))

In [ ]:
for img in tqdm(tt_4):
    shutil.copyfile(os.path.join(test_dir, img), os.path.join(tt_nm, img))

In [ ]:
for img in tqdm(tt_5):
    shutil.copyfile(os.path.join(test_dir, img), os.path.join(tt_nm, img))

In [ ]:
for img in tqdm(tt_6):
    shutil.copyfile(os.path.join(test_dir, img), os.path.join(tt_nm, img))

In [ ]:
for img in tqdm(tt_7):
    shutil.copyfile(os.path.join(test_dir, img), os.path.join(tt_nm, img))

In [ ]:
for img in tqdm(tt_8):
    shutil.copyfile(os.path.join(test_dir, img), os.path.join(tt_nm, img))

In [ ]:
for img in tqdm(tt_9):
    shutil.copyfile(os.path.join(test_dir, img), os.path.join(tt_nm, img))

In [ ]:
for img in tqdm(tt_10):
    shutil.copyfile(os.path.join(test_dir, img), os.path.join(tt_nm, img))